## Multiple Contracts

To demonstrate how we can use multiple contracts, we will create a strategy that looks at overnight price gaps for 3 contracts
It chooses the name that has the largest percentage overnight gap and goes long that name. The strategy exits by the end of the day
If price moves against us, we use a stop loss to get out. We use 1 minute bars for AAPL, NVDA and IBM

In [1]:
# %%checkall
import pandas as pd
import math
import pyqstrat as pq
import numpy as np
from collections import defaultdict
from dataclasses import dataclass
from types import SimpleNamespace
from typing import Sequence
from dataclasses import dataclass

_logger = pq.get_child_logger(__name__)


@dataclass
class OvernightReturn:
    name: str
    on_ret: float  # overnight return
    timestamps: np.ndarray
    prices: np.ndarray

    
def create_overnight_returns(contracts: list[str]) -> dict[np.datetime64, OvernightReturn]:
    on_rets: dict[np.datetime64, OvernightReturn] = {}
    for name in contract_names:
        filename = pq.find_in_subdir('.', f'{name}.csv.gz')
        prices = pd.read_csv(filename, usecols=['timestamp', 'c'])
        prices.timestamp = pd.to_datetime(prices.timestamp)
        prices['date'] = prices.timestamp.values.astype('M8[D]')
        prices['on_ret'] = np.where(prices.date > prices.date.shift(1), prices.c / prices.c.shift(1) - 1, np.nan)
        date_rets = prices[np.isfinite(prices.on_ret) & (prices.on_ret > 0)]
        dates = date_rets.date.values.astype('M8[D]')
        on_ret = date_rets.on_ret.values
        for i, date in enumerate(dates):
            if date not in on_rets or on_ret[i] > on_rets[date].on_ret:
                date_prices = prices[prices.date == date]
                on_rets[date] = OvernightReturn(name, on_ret[i], date_prices.timestamp.values.astype('M8[m]'), date_prices.c.values)
    return on_rets


def create_price_dataframe(on_rets: dict[np.datetime64, OvernightReturn]) -> pd.DataFrame:
    dfs: list[pd.DataFrame] = []
    for date, on_ret in on_rets.items():
        df = pd.DataFrame({'timestamp': on_ret.timestamps, 'price': on_ret.prices})
        df['date'] = df.timestamp.values.astype('M8[D]')
        df['contract'] = on_ret.name
        df['stop_price'] = df.groupby('date').price.first()
        dfs.append(df)
    prices = pd.concat(dfs)
    prices = prices[['timestamp', 'contract', 'date', 'price']]  # re-order columns
    prices = prices.sort_values(by=['timestamp'])
    return prices


def add_signals(prices: pd.DataFrame) -> pd.DataFrame:
    first = prices.sort_values(by=['timestamp']).drop_duplicates(subset=['contract', 'date'])
    first['enter'] = True
    first['stop_ret'] = -0.01
    first = first[['timestamp', 'contract', 'stop_ret', 'enter']]
    
    prices = pd.merge(prices, first, on=['timestamp', 'contract'], how='left')
    prices['stop_ret'] = prices.stop_ret.ffill()
    prices['stop'] = (prices.price <= prices.stop_price) 
    prices.enter = prices.enter.fillna(False)
    prices['eod'] = np.where(prices.date.shift(-2) > prices.date, True, False)   
    return prices


def create_price_function(on_rets: dict[np.datetime64, OvernightReturn]) -> pq.PriceFunctionType:
    price_dict: dict[str, dict[np.datetime64, float]] = defaultdict(dict)
    for on_ret in on_rets.values():
        price_dict[on_ret.name].update({on_ret.timestamps[i]: on_ret.prices[i] for i in range(len(on_ret.timestamps))})
    price_function = pq.PriceFuncDict(price_dict=price_dict)
    return price_function


@dataclass
class ContractFilter:
    '''
    For each day we want to trade only one symbol. So don't allow trade entry for all others
    '''
    def __init__(self, entry_contracts: dict[np.datetime64, list[str]]) -> None:
        self.entry_contracts = entry_contracts
        
    def __call__(self,
                 contract_group: pq.ContractGroup,
                 i: int,
                 timestamps: np.ndarray,
                 indicator_values: SimpleNamespace,
                 signal_values: np.ndarray,
                 account: pq.Account,
                 current_orders: Sequence[pq.Order],
                 strategy_context: pq.StrategyContextType) -> list[str]:
        date = timestamps[i].astype('M8[D]')
        return self.entry_contracts[date]


if __name__ == '__main__':
    contract_names = ['AAPL', 'NVDA', 'IBM']
    on_rets: dict[np.datetime64, OvernightReturn] = create_overnight_returns(contract_names)
    prices = create_price_dataframe(on_rets)
    prices = add_signals(prices)

    strat_builder = pq.StrategyBuilder(data=prices)
    for contract in contract_names:
        strat_builder.add_contract(contract)
    # add the stop price so we can refer to it in
    strat_builder.add_series_indicator('stop_price', 'stop_price') 
    price_function = create_price_function(on_rets)
    strat_builder.set_price_function(price_function)

    # This rule allows us to enter trades and get out with a limited loss when a stop is hit.
    entry_contracts = {date: on_ret.name for date, on_ret in on_rets.items()}
    entry_rule = pq.FiniteRiskEntryRule(
        reason_code='OVERNIGHT_RETURN',  # this is useful to know why we entered a trade
        contract_filter=ContractFilter(entry_contracts),
        price_func=price_function, 
        long=True,  # whether we enter a long or short position
        percent_of_equity=0.1,  # set the position size so that if the stop is hit, we lose no more than this
        # stop price is used for position sizing.  Also, we will not enter if the price is already below 
        # stop price for long trades and vice versa
        stop_price_ind='stop_price',
        single_entry_per_day=True)  # if we are stopped out, do we allow re-entry later in the day

    # ClosePositionExitRule fully exits a position using either a market or limit order
    # In this case, we want to exit at EOD so we are flat overnight
    exit_rule_stop = pq.ClosePositionExitRule(   
        reason_code='STOPPED_OUT',
        price_func=price_function)

    # Exit when the stop price is reached
    exit_rule_eod = pq.ClosePositionExitRule(
        reason_code='EOD',
        price_func=price_function)

    # Setup the rules we setup above so they are only called when the columns below in our data dataframe are true
    strat_builder.add_series_rule('enter', entry_rule, position_filter='zero')
    strat_builder.add_series_rule('eod', exit_rule_eod, position_filter='positive')
    strat_builder.add_series_rule('stop', exit_rule_stop, position_filter='positive')

    # create the strategy and run it
    strategy = strat_builder()
    strategy.run()


TypeError: FiniteRiskEntryRule.__init__() got an unexpected keyword argument 'stop_price_ind'

In [ ]:
# Lets look at the trades
strategy.df_trades()

In [ ]:
# Evaluate the returns
strategy.evaluate_returns(periods_per_year=252, plot=pq.has_display());

In [ ]:
strategy.df_roundtrip_trades()